## Scott Berry - Final Project Chatbot

Install dependencies

In [5]:
import random
import string
!pip install -q -U nltk
!pip install -q -U sklearn
!pip install -q -U pandas
!pip install -q -U numpy

Import dependencies

In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download("popular", quiet=True)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

Load clinical data into dataframe

In [20]:
df = pd.read_csv("clinical_data.csv")
raw_transcripts = pd.Series(df["transcription"].str.lower()).to_string()

Structure clinical data

In [8]:
sent_tokens = nltk.sent_tokenize(raw_transcripts)
word_tokens = nltk.word_tokenize(raw_transcripts)

lemmer = WordNetLemmatizer()

def lem_tokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def lem_normalize(text):
    return lem_tokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

AttributeError: 'numpy.ndarray' object has no attribute 'decode'

Write chatbot response logic

In [ ]:
def response(user_response):
    robot_response=""
    sent_tokens.append(user_response)
    tfidfvec = TfidfVectorizer(tokenizer=lem_normalize, stop_words="english")
    tfidf = tfidfvec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if req_tfidf == 0:
        robot_response += "I am still learning and did not understand you."
        return robot_response
    else:
        robot_response += sent_tokens[idx]
        return robot_response

Set chatbot greetings

In [ ]:
greeting_inputs = ["hello", "hi", "hey"]
greeting_response = ["Hello, how can I help you today?"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in greeting_inputs:
            return random.choice(greeting_response)

Chatbot prompts until bye condition filled

In [ ]:
print("Bot: Hello, I'm Scorberr, a chatbot of Scot Berry's creation. I'm an expert in clinical diagnoses, how can I assist you today?")
while True:
    user_response = input().lower()
    if user_response in ["bye", "goodbye"]:
        print("Until next time, goodbye!")
        break
    else:
        if greeting(user_response) is not None:
            print("Bot:", greeting(user_response))
        else:
            print("Bot:", response(user_response))
            sent_tokens.remove(user_response)